<a href="https://colab.research.google.com/github/anthonymelson/portfolio/blob/master/Constrained_Optimization_With_Scipy_Minimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Constrained Optimization With Scipy Minimize

## Import Packages

In [0]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


## Problem: Maximize the Volume of a Box While Keeping Surface Below 10 Units

Though this problem is a simple (toy) problem, it can be seen as representing a larger class of important problems.  Sticking with it as a representative, one may ask: whay would someone limit surface area and maximize volume?

The answer is that the surface space dictates the amount of materials used, and the volume (at least in some cases, like a box) represents the utility.  So, maximizing volume (utility), while lowering surface space (cost), is desirable.  
Additionally, these types of problems come up regularly, and can make a large difference in a project for those who know how to identify and solve them.

### Define Functions for Volume and Surface

Before the optimization problem can be fed to minimize, the geometric relationships between length, width, and highth must be defined.  The functions below accomplish this, and are in the form minimize expects them to be in.

Below, they will be related to the objective function and the constraint.  Where maximizing volume will be the primary objective (and called in the objective function), and surface (below 10 units) will be called by the constraint function and set as an inequality (which serves its purpose as long as the initial guesses are below that value to start).

In [0]:
#Define Box Volume and Surface Space
def calcVolume(x):
    l = x[0]
    w = x[1]
    h = x[2]
    vol = l * w * h
    return vol

def calcSurface(x):
    l = x[0]
    w = x[1]
    h = x[2]
    surf = 2*l*w + 2*l*h + 2*w*h
    return surf

### Define Objective Function

The objective function simply inputs the newest guesses for the variables from the minimize function at each round (this is done with x), and returns the negative value of the negative of the volume function evaluated at those points (negative because the negative of minimization is maximization, which is our goal)

In [0]:
#Define Objective Function    
def objective(x):
    return -calcVolume(x)

### Create the Surface Constraint

The constraint function calls the surface function evaluated at the set of points returned by x at each round of the optimization, and returns 10 minus that value (10 because it is our maximum allowable surface value).

However, for scipy to work properly, it must be put into a dictionary where its constraint type can be specified.  This is done with con1, and the constraint is set as an inequality.  Thus, the output must be unequal to 10 - surface area, which acts as an upper limit because the initial guesses evaluate below that mark.

In [0]:
#Create Constraint
def constraint1(x):
    return 10 - calcSurface(x)

con1 = {'type': 'ineq', 'fun': constraint1}

### Set Initial Guesses

The initial guesses are important for the optimizer to converge (find good solutions to the problem).  If they are too far off, it will terminate without success (convergence), and if they are on the wrong side of a constraint, the problem will be impossible to solve (as all the solutions will be in a space that is off limits).

Here the initial guesses are set at 1, 1, and 1, which is based on intuition alone in this case, but should be fine (as intuitions about simple things like boxes are often fairly accurate).

In [0]:
#Set Initial Guesses
x0 = np.array([1,1,1])

## Minimize the Function

The minimize function below takes four arguments for this problem:

1. objective (the objective function
2. x0 (the initial guesses for X1, X2, and X3 defined earlier)
3. method (SLSQP, the solver used to minimize the function)
4. constraints (con1, the constraint defined earlier)

Beyond that, it is a matter of letting it do its work.  If it converges, then the project is finished.  If not, it will require that the initial guesses are tweaked until a better result is found.

In [0]:
#Minimize Surface Function With SLSQP Solver, Constraint 1 in Effect and Initial Guesses for X1, X2, AND X3 set to x0

res = minimize(objective, x0, method='SLSQP', constraints=con1)

### Display Table of Results

In the table below:

* **X1, X2, and X3** are the optimal values for length, width, and hieght

* **Success** is either true or false and indicates whether the solver converged

* **Number of Iterations** is how many iterations it took to converge (or quit trying)

* **Cause of Determination** tells either that it converged or why it didn't.

In [54]:
#Create Results Table
results = {
            'X1': res.x[0],
            'X2': res.x[1],
            'X3': res.x[2],
            'Success': res.success,
            'Number of Iterations': res.nit,
            'Cause of Determination': res.message
                                                  }
display(pd.DataFrame(results, index=[1]).T)

,1
X1,1.29099
X2,1.29099
X3,1.29099
Success,True
Number of Iterations,4
Cause of Determination,Optimization terminated successfully.


### Results

These results are fantastic.  The optimization algorithm converged on a solution.  It turns out that a perfect cube does the job, but the question remains: how many units of volume and surface area did this produce?

To answer this, the volume function can be evaluated with the output points.

In [55]:
lwh = res.x

totals = {
          'Total Volume': calcVolume(lwh),
          'Total Surface Area': calcSurface(lwh)
                                                }
display(pd.DataFrame(totals, index=[1]).T)

,1
Total Volume,2.151657
Total Surface Area,10.000000


### Final Values

As expected, the total surface area is at the maximum allowable value given by the problem constraint (10 units).  This means that 2.151657 is almost the exact maximum volume acheivable within the constraint.